<h3> Create and Save a Barscan Calibration for Main and Wing detector</h3>

Most relevant function here is [calculate_barscan_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.calculate_barscan_calibration).

The output of <code>calculate_barscan_calibration</code> outputs an object containing two parts: a <b>metadata</b> (instrument name, day stamp, name of the double-detector-array), and the proper calibration data, contained in a Mantid <b>table</b> object.

In [ ]:
import numpy as np
import os
import time
from mantid.simpleapi import LoadEventNexus, LoadNexus
from drtsans.mono.biosans import calculate_barscan_calibration, plot_detector

#
# "plot_main_detector" and "plot_wing_detector" are used to plot both detectors separately
#
%matplotlib inline
def plot_main_detector(input_workspace, axes_mode='xy', panel_name='detector1'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_wing_detector(input_workspace, axes_mode='xy', panel_name='wing_detector'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_workspace(input_workspace, axes_mode='xy'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

The barscan files we'll use are runs 838 through 953. These runs have an embedded instrument incompatible with assumptions made by drtsans regarding the instrument definition file. I have loaded each file and overwritten the instrument with a newer version. After that I <b>integrated all events</b> in a given pixel into a single scalar representing the total number of counts. The processed data is saved onto files which are much quicker to load when calculating the calibration, and have an appropriate embedded file.

The script that I used for this task is commented below. It takes a long time to run and it's not necessary to run it again.

In [ ]:
"""
data_dir = '/HFIR/CG3/IPTS-23782/nexus/'
first_run, last_run = 838, 953
target_dir = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/runs_838_953'

for run in range(first_run, 1 + last_run):
    print('remaining files to process = ', 953 - run)
    file_path = os.path.join(data_dir, f'CG3_{run}.nxs.h5')
    LoadEventNexus(file_path, OutputWorkspace='scan', LoadNexusInstrumentXML=False)
    Rebin(InputWorkspace='scan', OutputWorkspace='scan', Params='0,100000,100000', PreserveEvents=False)
    outfile_path = os.path.join(target_dir, f'CG3_{run}.nxs')
    SaveNexus('scan', Filename=outfile_path)
"""

In [ ]:
data_dir = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/runs_838_953'
first_run, last_run = 838, 953
barscan_files = [os.path.join(data_dir, f'CG3_{run}.nxs') for run in range(first_run, 1 + last_run)]

Let's plot the middle scan, run 895, which is index 57 in the <code>barscan_files</code> list.

In [ ]:
LoadNexus(barscan_files[57], OutputWorkspace='CG3_895')
plot_workspace('CG3_895')

Notice the <b>last tube of the wing detector</b> is partically covered by the bar. Thus, we will <b>mask this tube</b> when calibrating the wing detector. We alread have a mask file for this purpose.

Now we carry out the calibration. Each detector array requires its own calibration. Notice we're using a custom formula `formula='{y} - 565'` that gives us the height of the bar with respect to the frame of reference of the same. Here, `{y}` is the value of log entry `dcal_Reaback`. This value is usually zero when the bar is at the top of the detector, so that the position of the bar in the frame of reference of the sample is `565 - {y}`. Then the value of `dcal_Readback` increases as the bar descends, reaching a maximum the bar reaches the bottom of the detetector. For this barscan, however, the value of `dcal_Readback` is maximum when the bar is at the top of the detector and minimum at the bottom. Thus, the formula we need to use is `'{y} - 565'` instead of the usual `565 - {y}`.
Everytime one runs a new calibration, the value of `dcal_Readback` <b>should be checked</b> when the bar is at the top of the detector to figure out which formula is the correct one.

In [ ]:
start_time = time.time()
calibration_main = calculate_barscan_calibration(barscan_files, component='detector1', formula='{y} - 565')
print('Calibration of the main detector array took ', int((time.time() - start_time) / 60), 'minutes')
calibration_main.save(overwrite=True)  # save calibration for the main detector

In [ ]:
mask_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/runs_838_953/biosans_mask_bank88_tube4.xml'
start_time = time.time()
calibration_wing = calculate_barscan_calibration(barscan_files, component='wing_detector', formula='{y} - 565', mask=mask_file)
print('Calibration of the wing detector array took ', int((time.time() - start_time) / 60), 'minutes')

<h5>Calibration objects are saved to a database as two separate pieces:</h5>

- metadata (instrument name, day stamp, name of the double-detector-array) is save to a JSON file.  
- data (a table workspace) is saved to a Nexus file with SaveNexus.  

There's a default database for every instrument. The BIOSANS location for the metadata JSON file:

- BIOSANS: <b>/HFIR/CG3/shared/calibration/pixel_calibration.json</b>

Data tables are saved under `tables/` subdirectory:

- BIOSANS: <b>/HFIR/CG3/shared/calibration/tables</b>

<h5>Calibration objects have method "<b>save</b>" to save itself to the the database. The full signature of this method:</h5>

    def save(self, database=None, tablefile=None, overwrite=False):
        r"""
        Save the calibration metadata in a JSON file, and the calibration table workspace in a Nexus file.

        Parameters
        ----------
        database: str
            Path to the JSON file where the ```metadata``` dictionary will be appended. If :py:obj:`None`,
            then the appropriate default file from ~drtsans.pixel_calibration.database_file is used.
            Currently, these are the default files:
            - BIOSANS, '/HFIR/CG3/shared/calibration/pixel_calibration.json',
            - EQSANS, '/SNS/EQSANS/shared/calibration/pixel_calibration.json',
            - GPSANS, '/HFIR/CG2/shared/calibration/pixel_calibration.json'
        tablefile: str
            Path to the Nexus file storing the pixel calibration data. If :py:obj:`None`, then
            a composite name is created using the calibration type, instrument, component,
            and daystamp. (e.g. "barscan_biosans_detector1_20200311"). The file is saved under
            subdirectory 'tables', located within the directory of the ```database``` file.
            For instance, '/HFIR/CG3/shared/calibration/tables/barscan_biosans_detector1_20200311.nxs'
        overwrite: bool
            Substitute existing entry with same metadata

        Raises
        ------
        ValueError
            If we save a calibration already in the database with option ```overwrite=False```.
        """

In [ ]:
# Notice we overwrite the already saved calibration, which will happen if we run this notebook more than once.
calibration_main.save(overwrite=True)  # save calibration for the main detector
calibration_wing.save(overwrite=True)  # save calibration for the main detector

<h3>Load and Apply a Barscan Calibration</h3>

Most relevant function is [load_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.load_calibration)

In [ ]:
import os
from mantid.simpleapi import LoadEventNexus
from drtsans.mono.biosans import load_calibration, plot_detector

#
# "plot_main_detector" and "plot_wing_detector" are used to plot both detectors separately
#
%matplotlib inline
def plot_main_detector(input_workspace, axes_mode='xy', panel_name='detector1'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_wing_detector(input_workspace, axes_mode='xy', panel_name='wing_detector'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_workspace(input_workspace, axes_mode='xy'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

Below we load the run to which we will apply the calibration. It is scan 58

We take a look by plotting the pixel intensities on both main and wing detectors.

In [ ]:
data_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/runs_838_953/CG3_895.nxs'
LoadNexus(data_file, OutputWorkspace='CG3_895')
plot_workspace('CG3_895')

Loading a calibration with [load_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.load_calibration) requires at least a target workspace and the type of calibration to be loaded. Function <code>load_calibration</code> returns a calibration object.

The calibration object has method <b>apply</b> to apply itself into a target workspace. Here's the full documentation for this method:

    def apply(self, input_workspace, output_workspace=None):
        r"""
        Apply a calibration to an input workspace, and return the calibrated workspace.

        Parameters
        ----------
        input_workspace: str, ~mantid.api.MatrixWorkspace, ~mantid.api.IEventsWorkspace
            Workspace to which calibration needs to be applied.
        output_workspace: str
            Name of the output workspace with calibrated pixels. If :py:obj:`None`, the pixels
            of the input workspace will be calibrated and no new workspace is generated.

        Returns
        -------
        ~mantid.api.MatrixWorkspace, ~mantid.api.IEventsWorkspace
        """

In [ ]:
calibration_database_main = load_calibration('CG3_895', 'BARSCAN', component='detector1')
calibration_database_wing = load_calibration('CG3_895', 'BARSCAN', component='wing_detector')
# Apply the calibration.
start_time = time.time()
calibration_database_main.apply('CG3_895')
calibration_database_wing.apply('CG3_895')
print(f'Applying the two calibrations took {time.time() - start_time:.1} seconds')

In [ ]:
plot_workspace('CG3_895', axes_mode='xy')

print('min position on main detector =', min(calibration_database_main.positions), 'm,',
      'max position on main detector =', max(calibration_database_main.positions), 'm\n')

print('min height on main detector =', 1000 * min(calibration_database_main.heights), 'mm,',
      'max height on main detector =', 1000 * max(calibration_database_main.heights), 'mm')

In [ ]:
views_main = calibration_database_main.as_intensities('CG3_895')

In [ ]:
plot_workspace(views_main.positions, axes_mode='tube-pixel')

In [ ]:
plot_workspace(views_main.heights, axes_mode='tube-pixel')

The main detector is not as properly calibrated as the wing detector. This is due to the <b>low events count</b> collected at the main detector (about ~8 counts per pixel) compared to the wing detector (about ~200 counts per pixel). The counts at the main dectector pixels should reach ~100.

<h3>Calculate Tube Width Calibration</h3>

Relevant function is [calculate_apparent_tube_width](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.calculate_apparent_tube_width)

In [ ]:
import numpy as np
import os
import time
from mantid.simpleapi import LoadNexus
from drtsans.mono.biosans import calculate_apparent_tube_width, plot_detector

#
# "plot_main_detector" and "plot_wing_detector" are used to plot both detectors separately
#
%matplotlib inline
def plot_main_detector(input_workspace, axes_mode='xy', panel_name='detector1'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_wing_detector(input_workspace, axes_mode='xy', panel_name='wing_detector'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})
def plot_workspace(input_workspace, axes_mode='xy'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

The flood file CG3_4829.nxs.h5 is 9.3GB in size but we only require the total intensity per pixel. We implement the following steps to convert into a simple Mantid MatrixWorkspace:

- `LoadEventNexus` to load the events file
- `Rebin` to summ all events into a single total count (per pixel)
- `SaveNexus` saves the MatrixWorkspace to a file.

Here's the script I run in another location. You can copy and paste into a jupyter notebook cell to run it.

    from mantid.simpleapi import LoadEventNexus, LoadInstrument, Rebin, SaveNexus

    flood_file = '/HFIR/CG3/IPTS-23782/nexus/CG3_4829.nxs.h5'
    LoadEventNexus(Filename=flood_file, OutputWorkspace='flood_workspace')
    Rebin(InputWorkspace='flood_workspace', OutputWorkspace='flood_workspace',
          Params=[0, 1.E06, 1.E06], PreserveEvents=False)
    flood_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/CG3_4829.nxs'
    SaveNexus(InputWorkspace='flood_workspace', Filename=flood_file)

The rebinned file is 178 MB in size.

In [ ]:
flood_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/CG3_4829.nxs'
LoadNexus(Filename=flood_file, OutputWorkspace='flood_workspace')

The main detector needs the beam center to be masked prior to running the tube width calibration:

In [ ]:
plot_main_detector('flood_workspace', axes_mode='tube-pixel')

We have masked the beam center and saved the mask in a file. Here we apply the mask to the workspace using `apply_mask`

In [ ]:
def apply_mask(input_workspace, mask=None, panel=None, output_workspace=None, **btp): r""" Apply a mask to a workspace.

The function accepts a path to a mask file, a MaskWorkspace, or options
to algorithm :ref:`MaskBTP <algm-MaskBTP-v1>`.

Parameters
----------
input_workspace: str, ~mantid.api.IEventWorkspace, ~mantid.api.MatrixWorkspace
    Workspace to be masked
mask: mask file path, ~mantid.api.MaskWorkspace, :py:obj:`list`
    Additional mask to be applied. If :py:obj:`list`, it is a list of
    detector ID's. If `None`, it is expected that `maskbtp` is not empty.
panel: str
    Either 'front' or 'back' to mask a whole panel
output_workspace: str
    Name of the output ~mantid.api.MatrixWorkspace. If ``None``, a random name will be provided for the workspace.
btp: dict
    Options to Mantid algorithm :ref:`MaskBTP <algm-MaskBTP-v1>` or :ref:`MaskAngle <algm-MaskAngle-v1>`.
    Will be used if  ``mask=None``

Returns
-------
MaskWorkspace
    Combination of panel, mask, and :ref:`MaskBTP <algm-MaskBTP-v1>` masks
"""

In [ ]:
from drtsans.mono.biosans import apply_mask
mask_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/mask_4829.xml'
apply_mask('flood_workspace', mask=mask_file)

In [ ]:
plot_main_detector('flood_workspace', axes_mode='tube-pixel')

Ideally, calculation of the apparent tube width requires that the pixel positions and heights have been calibrated with a barscan. If no good barscan is present in the database, we can use the default pixel positions and heights defined in the instrument defintion file by setting <code>load_barscan_calibration=False</code>

In [ ]:
start_time = time.time()
calibration_main = calculate_apparent_tube_width('flood_workspace', component='detector1',
                                                 load_barscan_calibration=False)
calibration_wing = calculate_apparent_tube_width('flood_workspace', component='wing_detector',
                                                 load_barscan_calibration=False)
print('Calibration took ', int(time.time() - start_time), 'seconds')

<h5>Calibration objects are saved to a database as two separate pieces:</h5>

- metadata (instrument name, day stamp, name of the double-detector-array) is save to a JSON file.  
- data (a table workspace) is saved to a Nexus file with SaveNexus.  

There's a default database for every instrument. The BIOSANS location for the metadata JSON file:

- BIOSANS: <b>/HFIR/CG3/shared/calibration/pixel_calibration.json</b>

Data tables are saved under `tables/` subdirectory:

- BIOSANS: <b>/HFIR/CG3/shared/calibration/tables</b>

<h5>Calibration objects have method "<b>save</b>" to save itself to the the database. The full signature of this method:</h5>

    def save(self, database=None, tablefile=None, overwrite=False):
        r"""
        Save the calibration metadata in a JSON file, and the calibration table workspace in a Nexus file.

        Parameters
        ----------
        database: str
            Path to the JSON file where the ```metadata``` dictionary will be appended. If :py:obj:`None`,
            then the appropriate default file from ~drtsans.pixel_calibration.database_file is used.
            Currently, these are the default files:
            - BIOSANS, '/HFIR/CG3/shared/calibration/pixel_calibration.json',
            - EQSANS, '/SNS/EQSANS/shared/calibration/pixel_calibration.json',
            - GPSANS, '/HFIR/CG2/shared/calibration/pixel_calibration.json'
        tablefile: str
            Path to the Nexus file storing the pixel calibration data. If :py:obj:`None`, then
            a composite name is created using the calibration type, instrument, component,
            and daystamp. (e.g. "barscan_biosans_detector1_20200311"). The file is saved under
            subdirectory 'tables', located within the directory of the ```database``` file.
            For instance, '/HFIR/CG3/shared/calibration/tables/barscan_biosans_detector1_20200311.nxs'
        overwrite: bool
            Substitute existing entry with same metadata

        Raises
        ------
        ValueError
            If we save a calibration already in the database with option ```overwrite=False```.
        """

In [ ]:
# Notice we overwrite the already saved calibration, which will happen if we run this notebook more than once.
calibration_main.save(overwrite=True)
calibration_wing.save(overwrite=True)

<h3>Load and Apply a Tube Width Calibration</h3>

First some general imports and a couple of custom plotting functions

In [ ]:
import time
from mantid.api import mtd
from mantid.simpleapi import CreateWorkspace, LoadNexus
from drtsans.tubecollection import TubeCollection
from matplotlib import pyplot as plt
#
# "plot_histograms" to create fancy plots of the spectram stored in an input workspace
#
def plot_histograms(input_workspace, legend=[], xlabel='X-axis', ylabel='Y-axis', title='', linewidths=[]):
    r"""Line plot for the histograms of a workspace"""
    workspace = mtd[str(input_workspace)]
    number_histograms = workspace.getNumberHistograms()
    if len(legend) != number_histograms:
        legend = [str(i) for i in range(number_histograms)]
    if len(linewidths) != number_histograms:
        linewidths = [1] * number_histograms
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    for workspace_index in range(number_histograms):
        ax.plot(workspace, wkspIndex=workspace_index, label=legend[workspace_index],
                linewidth=linewidths[workspace_index])
    ax.legend()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis='x', direction='in')
    ax.tick_params(axis='y', direction='out')
    ax.grid(True)
    fig.show()

In [ ]:
# Load the run to which we will apply the calibration, then plot the pixel intensities
input_data = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/CG3_4829.nxs'
LoadNexus(Filename=input_data, OutputWorkspace='workspace')

In [ ]:
plot_workspace('workspace', axes_mode='tube-pixel')

The run corresponds to a flood run.

We have masked the beam center and saved the mask in a file. Here we apply the mask to the workspace using `apply_mask`

In [ ]:
from drtsans.mono.biosans import apply_mask
mask_file = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/mask_4829.xml'
apply_mask('workspace', mask=mask_file)

In [ ]:
plot_main_detector('workspace', axes_mode='tube-pixel')

<h4>Intensities Normalized by Pixel Width</h4>
In function <code>linear_density</code> we integrate the total intensity per tube and divide by the number of non-masked pixels in the tube, and by the tube width. Front end tubes collect more intentity than the back tubes. Similarly, front end tubes have a larger apparent tube width than back tubes. The ratio of total intensity to width should be similar for front and end tubes after the calibration.

In [ ]:
def linear_density(workspace, component='detector1'):
    r"""Tube total intensity per unit length of tube width"""
    collection = TubeCollection(workspace, component).sorted(view='decreasing X')
    intensities = np.array([np.sum(tube.readY) for tube in collection])
    widths = np.array([tube.width for tube in collection])
    number_pixels_not_masked = np.array([np.sum(~tube.isMasked) for tube in collection])
    return list(intensities / (number_pixels_not_masked * widths))

uncalibrated_main_densities = linear_density('workspace', component='detector1')
uncalibrated_wing_densities = linear_density('workspace', component='wing_detector')

We store both linear densities in a workspace, and then we'll use matplotlib to plot both densities.

Next, we load and apply the calibration to each detector. The relevant function is [load_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.load_calibration)

In [ ]:
from drtsans.mono.biosans import load_calibration
start_time = time.time()
calibration_main = load_calibration('workspace', 'TUBEWIDTH', component='detector1')
calibration_main.apply('workspace')

calibration_wing = load_calibration('workspace', 'TUBEWIDTH', component='wing_detector')
calibration_wing.apply('workspace')
print('Applying the calibration took ', time.time() - start_time, 'seconds')

calibrated_main_densities = linear_density('workspace', component='detector1')
calibrated_wing_densities = linear_density('workspace', component='wing_detector')

We store both linear densities in a workspace, and then we'll use matplotlib to plot both densities

In [ ]:
number_tubes = len(uncalibrated_main_densities)
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([uncalibrated_main_densities, calibrated_main_densities]),
                NSpec=2,   # two histograms
                Outputworkspace='main_linear_densities')
plot_histograms('main_linear_densities', title='Main Detector Linear Densities',
                legend=['no calibration', 'calibrated'],
                xlabel='Tube Index', ylabel='Intensity', linewidths=[3, 1])

number_tubes = len(uncalibrated_wing_densities)
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([uncalibrated_wing_densities, calibrated_wing_densities]),
                NSpec=2,   # two histograms
                Outputworkspace='wing_linear_densities')
plot_histograms('wing_linear_densities', title='Wing Detector Linear Densities',
                legend=['no calibration', 'calibrated'],
                xlabel='Tube Index', ylabel='Intensity', linewidths=[3, 1])

The oscillating intensities in the linear densities have been suppresed for most tubes, indicating the calibration is working.

<h3>Loading and Applying a pixel calibration</h3>

Relevant function is [apply_calibrations](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.apply_calibrations), which will search for <code>BARSCAN</code> and <code>TUBEWIDTH</code> calibrations appropriate to the target run

In [ ]:
import time
from mantid.simpleapi import LoadNexus
from drtsans.mono.biosans import apply_calibrations

#
# "plot_workspace" is a utility function, which we will use a couple of times
#
from drtsans.plots import plot_detector
def plot_workspace(input_workspace, axes_mode='tube-pixel'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

In [ ]:
# Load the run to which we will apply the calibration, then plot the pixel intensities
input_data = '/HFIR/CG3/shared/sans-backend/data/new/ornl/sans/hfir/biosans/pixel_calibration/flood_files/CG3_4829.nxs'
LoadNexus(Filename=input_data, OutputWorkspace='workspace')

plot_workspace('workspace', axes_mode='xy')

In [ ]:
start_time = time.time()
apply_calibrations('workspace')
print('Applying the calibration took ', time.time() - start_time, 'seconds')

In [ ]:
plot_workspace('workspace', axes_mode='xy')

We can see when the BARSCAN calibration was taken, and the time when the input data was taken. For this we use function [load_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.load_calibration) and function <code>day_stamp</code>.

In [ ]:
from drtsans.pixel_calibration import load_calibration, day_stamp
barscan_calibration = load_calibration('workspace', 'BARSCAN')
tubewidth_calibration = load_calibration('workspace', 'TUBEWIDTH')
print('BARSCAN taken on', barscan_calibration.daystamp)
print('TUBEWIDTH taken on', tubewidth_calibration.daystamp)
print('Input data taken on ', day_stamp('workspace'))